In [ ]:
# Install spacy on NLU environment
# > conda install -c conda-forge spacy 
# > python -m spacy download en_core_web_md

In [1]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

nlp = en_core_web_sm.load()

In [8]:
import pandas as pd
import dataset
import vsm
import sst

In [18]:
from pprint import pprint


In [2]:
TWITTER = 2
TWITTER_AIRLINES = 3
TWITTER_APPLE = 4

------------

## Exploring Apple Dataset

In [5]:
twitter_train, twitter_validate, twitter_test =  dataset.dataset_reader(TWITTER_APPLE)
[twitter_train, twitter_validate, twitter_test] = list(map(lambda ds : dataset.prune_columns(TWITTER_APPLE, ds), [twitter_train, twitter_validate, twitter_test]))

In [6]:
[ds.shape for ds in [twitter_train, twitter_validate, twitter_test]]

[(3109, 4), (388, 4), (389, 4)]

In [9]:
twitter_ds = pd.concat([twitter_train,twitter_validate,twitter_test],axis=0)

In [10]:
twitter_ds

,dataset,tweet_id,text,sentiment
1408,twitter_apple,623496932,RT @huffpostgay: @Apple CEO Time Cook gets a u...,3
3132,twitter_apple,623498662,NO CALLER ID must be able to be BLOCKED on @Ap...,1
1362,twitter_apple,623496886,RT @JPDesloges: iPhone 6 makes iOS a success i...,3
988,twitter_apple,623496506,RT @HamzeiAnalytics: BLOCK TRADE detected in #...,3
1516,twitter_apple,623497040,@OneRepublic @Apple Lmaoo! Someone needs to ge...,3
...,...,...,...,...
3810,twitter_apple,623499340,We hope @Apple doesn't use glass for their new...,1
3812,twitter_apple,623499342,@Apple co-founder Steve Wozniak talks about St...,3
3840,twitter_apple,623499370,Lets go shopping!!! @apple http://t.co/uTFUHuoJIi,5
3847,twitter_apple,623499377,proof @apple does not use it's own products. f...,1


### Exploring single text input

In [16]:
text = twitter_ds['text'][6] #6 -> tablets
text

'Top 3 all @Apple #tablets. Damn right! http://t.co/RJiGn2JUuB'

In [17]:
doc = nlp(text)
doc

Top 3 all @Apple #tablets. Damn right! http://t.co/RJiGn2JUuB

In [19]:
pprint([(X.text, X.label_) for X in doc.ents])

[('3', 'CARDINAL')]


### Exploring all tweets entities

In [21]:
twitter_ds['ner_entities'] = twitter_ds['text'].apply(lambda x: [(ent.text, ent.label_) for ent in nlp(x).ents])

In [22]:
twitter_ds['ner_entities'] 

1408    [(Time Cook, PERSON), (Alabama http://t.co/TDR...
3132                [(BLOCKED, ORG), (#iPhone6 #, MONEY)]
1362    [(6, CARDINAL), (UK, GPE), (Android, ORG), (Ja...
988                                                    []
1516                                                   []
                              ...                        
3810    [(Williamsburg, PERSON), (Sikrikim, PERSON), (...
3812    [(Steve Wozniak, PERSON), (Steve Jobs &amp, OR...
3840                                   [(@apple, PERSON)]
3847                                        [(MAIL, ORG)]
3884                                                   []
Name: ner_entities, Length: 3886, dtype: object

Get only records with PRODUCT labels:

In [27]:
def find_product(ents, tag="PRODUCT"):    
    entities = set()
    [entities.add(tup[1]) for tup in ents]    
    return tag in entities
# test
find_product([('6', 'CARDINAL'), ('UK', 'GPE'), ('Android', 'ORG'), ('x','PRODUCT')])

True

In [31]:
twitter_products_ds = twitter_ds[twitter_ds.ner_entities.apply(lambda x: find_product(x))]
twitter_products_ds.shape

(60, 5)

In [32]:
twitter_products_ds.head()

,dataset,tweet_id,text,sentiment,ner_entities
178,twitter_apple,623495691,None too happy with @Apple Mac OS X (Yosemite)...,1,"[(@Apple Mac OS, ORG), (Yosemite, NORP), (File..."
2355,twitter_apple,623497885,AFP requests to Apple reveal 'get a warrant' p...,3,"[(AFP, ORG), (Apple, ORG), (http://t.co/7zH5iM..."
860,twitter_apple,623496373,#New #foldable #Macbook #Pro falling on your l...,3,"[(Pro, PERSON), (@Apple s, PRODUCT)]"
3069,twitter_apple,623498599,"In the @Apple iPod antitrust case, recent deve...",3,"[(@Apple iPod, ORG), (iPods, PRODUCT), (first,..."
3707,twitter_apple,623499237,CNBCTV: #Cramer's view on #Apple &amp; Tesla ...,3,"[(Cramer, PERSON), (Apple &amp, ORG), (Tesla, ..."


--------------
--------------

## NER + Sentiment

### Get Entities and Product Flag into Sentiment Analysis Results (Twitter Apple Dataset)

In [56]:
apple_sent = pd.read_csv('results/BERTweet_predictions_added_to_twitter_test_apple.csv',index_col=0)

In [57]:
apple_sent.head()

,dataset,tweet_id,text,sentiment,BERTweet_sentiment
12,twitter_apple,623495525,"The Best-Designed #iPhone #Apps In the World, ...",3,3
15,twitter_apple,623495528,#aapl @applenws Thanks to the non factual dumb...,3,3
38,twitter_apple,623495551,RT @thehill: Justice Department cites 18th cen...,3,3
43,twitter_apple,623495556,RT @thehill: Justice Department cites 18th cen...,1,3
45,twitter_apple,623495558,RT @thehill: Justice Department cites 18th cen...,3,3


In [58]:
def get_entities(ds_text_col):    
    return ds_text_col.apply(lambda x: [(ent.text, ent.label_) for ent in nlp(x).ents])
    
def get_product_flag(ds_entity_col):
    def find_product(ents, tag="PRODUCT"):    
        entities = set()
        [entities.add(tup[1]) for tup in ents]            
        return tag in entities
    return ds_entity_col.apply(lambda x: find_product(x))    

In [59]:
apple_sent['NER_ents'] = get_entities(apple_sent['text'])

In [60]:
apple_sent['NER_prod_flag']  = get_product_flag(apple_sent['NER_ents'])

In [61]:
apple_sent

,dataset,tweet_id,text,sentiment,BERTweet_sentiment,NER_ents,NER_prod_flag
12,twitter_apple,623495525,"The Best-Designed #iPhone #Apps In the World, ...",3,3,"[(Apps, PERSON)]",False
15,twitter_apple,623495528,#aapl @applenws Thanks to the non factual dumb...,3,3,"[(aapl @applenws, PERSON), (3, CARDINAL), (one...",False
38,twitter_apple,623495551,RT @thehill: Justice Department cites 18th cen...,3,3,"[(Justice Department, ORG), (18th century, DAT...",False
43,twitter_apple,623495556,RT @thehill: Justice Department cites 18th cen...,1,3,"[(Justice Department, ORG), (18th century, DAT...",False
45,twitter_apple,623495558,RT @thehill: Justice Department cites 18th cen...,3,3,"[(Justice Department, ORG), (18th century, DAT...",False
...,...,...,...,...,...,...,...
3853,twitter_apple,623499383,So @Apple just gave me a huge middle finger to...,1,1,[],False
3869,twitter_apple,623499399,Apple Is Warming Up To Social Media: Apple is ...,3,3,"[(Apple, ORG), (Apple, ORG), (L.A., GPE)]",False
3870,twitter_apple,623499400,Apple Is Warming Up To Social Media: Apple is ...,3,3,"[(Apple, ORG), (Apple, ORG), (L.A., GPE)]",False
3874,twitter_apple,623499404,Apple Is Warming Up To Social Media: Apple is ...,3,3,"[(Apple, ORG), (Apple, ORG), (L.A., GPE)]",False


Get tweets where NER detected PRODUCT tag:

In [63]:
apple_products = apple_sent[apple_sent.NER_prod_flag]
apple_products

,dataset,tweet_id,text,sentiment,BERTweet_sentiment,NER_ents,NER_prod_flag
178,twitter_apple,623495691,None too happy with @Apple Mac OS X (Yosemite)...,1,1,"[(@Apple Mac OS, ORG), (Yosemite, NORP), (File...",True
649,twitter_apple,623496162,That would be brilliant! It's sad @Apple doesn...,1,1,"[(@Apple, ORG), (Mozilla, PRODUCT), (@OpenSour...",True
1155,twitter_apple,623496674,@Apple removed songs from iPods without tellin...,1,3,"[(@Apple, ORG), (iPods, PRODUCT)]",True
1790,twitter_apple,623497314,#AAPL:Apple says plaintiffs' iPods not covered...,3,3,"[(AAPL, PERSON), (Apple, ORG), (iPods, PRODUCT)]",True


In [75]:
def get_sent(sent):
    if sent == '1':
        return "negative"
    elif sent == '3':
        return "neutral"
    else:
        return "positive"
for sentiment, tweet in zip(apple_products['sentiment'], apple_products['text']):     
    print('[',get_sent(sentiment),'] ->',tweet,'\n--------------------------')

[ negative ] -> None too happy with @Apple Mac OS X (Yosemite), File Vault 2, or Boot Camp right now. Thankful to own a @lenovo. 
--------------------------
[ negative ] -> That would be brilliant! It's sad @Apple doesn't allow Mozilla's better engine @OpenSourceAgent #opensource 
--------------------------
[ negative ] -> @Apple removed songs from iPods without telling customers http://t.co/C29jxFuNcJ 
--------------------------
[ neutral ] -> #AAPL:Apple says plaintiffs' iPods not covered by suit...http://t.co/8V0eYzQFDQ 
--------------------------


-----

### Get Entities and Product Flag into Sentiment Analysis Results (Airline Datasets)

In [82]:
df = pd.read_csv('results/BERTweet_predictions_added_to_twitter_test_airline.csv',index_col=0)
df.head()

,dataset,tweet_id,text,sentiment,airline,BERTweet_sentiment
5,twitter_airline,570300767074181121,@VirginAmerica seriously would pay $30 a fligh...,negative,Virgin America,negative
18,twitter_airline,570270684619923457,I ❤️ flying @VirginAmerica. ☺️👍,positive,Virgin America,positive
24,twitter_airline,570256553502068736,@VirginAmerica you guys messed up my seating.....,negative,Virgin America,negative
48,twitter_airline,570010539499393025,@VirginAmerica @ladygaga @carrieunderwood Afte...,neutral,Virgin America,positive
52,twitter_airline,570004391731847169,@VirginAmerica wish you flew out of Atlanta......,neutral,Virgin America,neutral


In [80]:
df['text'].astype(str)

2       im getting on borderlands and i will kill you ...
3       im coming on borderlands and i will murder you...
7       So I spent a couple of hours doing something f...
14      Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...
17      I-Hard like me, RARE LONDON DE, HANDSOME 2011,...
                              ...                        
3836    @Apple recruiting luxury executives for iwatch...
3846    RT @TeamCavuto: Protesters stage #DieIn protes...
3858    Apple Is Warming Up To Social Media: Apple is ...
3870    Apple Is Warming Up To Social Media: Apple is ...
3872    Apple Is Warming Up To Social Media: Apple is ...
Name: text, Length: 24258, dtype: object

In [83]:
df['NER_ents'] = get_entities(df['text'])

In [84]:
df['NER_prod_flag']  = get_product_flag(df['NER_ents'])

In [85]:
df

,dataset,tweet_id,text,sentiment,airline,BERTweet_sentiment,NER_ents,NER_prod_flag
5,twitter_airline,570300767074181121,@VirginAmerica seriously would pay $30 a fligh...,negative,Virgin America,negative,"[(@VirginAmerica, GPE), (30, MONEY), (VA, ORG)]",False
18,twitter_airline,570270684619923457,I ❤️ flying @VirginAmerica. ☺️👍,positive,Virgin America,positive,"[(@VirginAmerica, GPE), (☺, ORG)]",False
24,twitter_airline,570256553502068736,@VirginAmerica you guys messed up my seating.....,negative,Virgin America,negative,"[(@VirginAmerica, GPE)]",False
48,twitter_airline,570010539499393025,@VirginAmerica @ladygaga @carrieunderwood Afte...,neutral,Virgin America,positive,"[(@VirginAmerica @ladygaga @carrieunderwood, O...",False
52,twitter_airline,570004391731847169,@VirginAmerica wish you flew out of Atlanta......,neutral,Virgin America,neutral,"[(@VirginAmerica, GPE), (Atlanta, GPE)]",False
...,...,...,...,...,...,...,...,...
14595,twitter_airline,569593694963310593,@AmericanAir @ShannonBloom Where's my DM? Wher...,negative,American,neutral,"[(70, MONEY), (50, MONEY), (JFK, PERSON), (tom...",False
14607,twitter_airline,569592270866878464,@AmericanAir i need someone to help me out,neutral,American,negative,[],False
14614,twitter_airline,569591540944756737,@AmericanAir I need to be at work tomorrow at ...,negative,American,negative,"[(tomorrow, DATE), (8am, TIME), (800, CARDINAL...",False
14620,twitter_airline,569590965880532993,@AmericanAir I wait 2+ hrs for CS to call me b...,negative,American,negative,"[(2, CARDINAL), (CS, ORG), (the minute, TIME),...",False


In [88]:
df_products = df[df.NER_prod_flag]
df_products.shape

(40, 8)

In [90]:
for sentiment, tweet in zip(df_products['sentiment'], df_products['text']):     
    print('[',sentiment,'] ->',tweet,'\n--------------------------')

[ neutral ] -> @VirginAmerica Can't bring up my reservation online using Flight Booking Problems code 
--------------------------
[ negative ] -> @VirginAmerica And now the flight Flight Booking Problems site is totally down. Folks, what is the problem? 
--------------------------
[ positive ] -> @VirginAmerica thanks to your outstanding NYC-JFK crew who moved mountains to get me home to San Francisco tonight! 
--------------------------
[ neutral ] -> @VirginAmerica - can you tweet me the Cancelled Flight/chng fee for a flight? or can I rebook under one of your affiliates? If so, who are afiliates? 
--------------------------
[ neutral ] -> @united Left item n the seatback on UA1260. Is there any way to call DCA to ask if they have the item? Already submitted lost &amp; found report 
--------------------------
[ neutral ] -> @united Seat 14A, Flight UA895 
--------------------------
[ neutral ] -> @united any chance you'll ever do CPUs on your JFK-LAX like @AmericanAir? 
-------------

### Get Entities and Product Flag into Sentiment Analysis Results (Twitter Datasets)

In [91]:
df = pd.read_csv('results/BERTweet_predictions_added_to_twitter_test.csv',index_col=0)
df.head()


,dataset,tweet_id,text,sentiment,entity,BERTweet_sentiment
3,twitter_sentiment,1,im coming on borderlands and i will murder you...,Positive,2401,Positive
7,twitter_sentiment,2,So I spent a couple of hours doing something f...,Positive,2402,Neutral
8,twitter_sentiment,3,So I spent a few hours doing something for fun...,Positive,2402,Positive
12,twitter_sentiment,4,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...",Neutral,2403,Neutral
13,twitter_sentiment,5,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...",Neutral,2403,Neutral


In [94]:
df['text'] = df['text'].astype(str)

In [95]:
df['NER_ents'] = get_entities(df['text'])
df['NER_prod_flag']  = get_product_flag(df['NER_ents'])
df

,dataset,tweet_id,text,sentiment,entity,BERTweet_sentiment,NER_ents,NER_prod_flag
3,twitter_sentiment,1,im coming on borderlands and i will murder you...,Positive,2401,Positive,[],False
7,twitter_sentiment,2,So I spent a couple of hours doing something f...,Positive,2402,Neutral,"[(a couple of hours, TIME), (Maya, PERSON)]",False
8,twitter_sentiment,3,So I spent a few hours doing something for fun...,Positive,2402,Positive,"[(a few hours, TIME), (HUGE, ORG), (Maya, PERS...",False
12,twitter_sentiment,4,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...",Neutral,2403,Neutral,"[(Rock-Hard La Varlope, ORG), (RARE & POWERFUL...",False
13,twitter_sentiment,5,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...",Neutral,2403,Neutral,"[(Rock-Hard La Varlope, ORG), (RARE & POWERFUL...",False
...,...,...,...,...,...,...,...,...
74657,twitter_sentiment,22401,really doesn't look bad btw!.. net.com/google-...,Negative,9196,Neutral,[],False
74661,twitter_sentiment,22402,Nvidia doesn’t intend to give away its 2017 ad...,Neutral,9197,Neutral,"[(Nvidia, ORG), (2017, DATE), (docs maxbit.cc/...",False
74662,twitter_sentiment,22403,Nvidia therefore doesn ’ t want to give up its...,Neutral,9197,Neutral,"[(Nvidia, GPE), (2017, DATE), (4 cc / 9, QUANT...",False
74671,twitter_sentiment,22404,t let Elim go unnoticed.... NVIDIA Highlights ...,Positive,9199,Neutral,"[(Elim, PERSON), (FortniteGame, ORG)]",False


In [98]:
f_name ='ner_results/BERTweet_twitter_sentiment_ner.csv'
df.to_csv(f_name)

In [96]:
df_products = df[df.NER_prod_flag]
df_products.shape

(893, 8)

In [97]:
for sentiment, tweet in zip(df_products['sentiment'], df_products['text']):     
    print('[',sentiment,'] ->',tweet,'\n--------------------------')

[ Positive ] -> Ok I am LOVING THIS (also playing Bombs and Earth in anything gets me hype) Very good RPG and the concept is A1! 
--------------------------
[ Neutral ] -> For some reason my laptop can run Borderlands 3, War Thunder and Warframe at high settings... BUT.. the moment I even try to start Csgo, my entire computer crashes... 
--------------------------
[ Neutral ] -> For some reason, my Firefox can run Borderlands 3, War thunder plus Warframe all on high settings... no.. The moment I even try to even launch Csgo its entire computer crashes... 
--------------------------
[ Neutral ] -> For some reason, my laptop can easily run Borderlands Volume 3, Chaos War thunder and Warframe all currently on high settings... A BUT.. A The moment I even dare try to even launch Csgo to my mind entire computer crashes... 
--------------------------
[ Positive ] -> I am here... With Samsung that is full of homeworks. Away from ps4 and Samsung that we play fort. And have 1700... Want psycho. 

[ Negative ] -> NBA 2K is the dustiest sports game franchise in the universe. . . They are so trash bruh lmaooo bruh 2K used to be fun as hell back in the day. These niggas fucked the whole game up. 
--------------------------
[ Irrelevant ] -> Pissed off the NBA 2K community today lol — these dumbass beta boys enjoy shelling out all this money for this broken game that’s stealing their money. 
--------------------------
[ Irrelevant ] -> Pissed off at the NBA 2K community today by lol all... these dumbass old fan boy boys enjoy shelling out all this stupid money for hosting this broken game that ’ s stealing your money. 
--------------------------
[ Positive ] -> Caw is dead, everything we know is NBA 2K now. 
--------------------------
[ Positive ] -> Caw is dead, all hope we know is no NBA 2K now. 
--------------------------
[ Positive ] -> Caw is born, that we know is NBA 2K now. 
--------------------------
[ Negative ] -> MyTeam Man is stupid. Why are the packs so bad, why is ther

[ Positive ] -> Ik y'all don't really expect a Lamborghini, but I really want this PS5. 
--------------------------
[ Neutral ] -> also if my man can’t buy his own PS5.....throw that bitch away 
--------------------------
[ Neutral ] -> even if my husband can't buy his own PS5..... throw this bitch away 
--------------------------
[ Neutral ] -> also if my man can’t buy his own PS5.....throw it bitch away 
--------------------------
[ Positive ] -> With the announcement of the PS5, I have to say... With prices I've seen from both Xbox nor MS, I really and't wait to get that 3080 for my PC. 
--------------------------
[ Positive ] -> Aye if another X-box or PS5 preorder becomes available plz send it to my phone (or mentions) as soon as u find out please. If u fuck with me. 
--------------------------
[ Positive ] -> Aye if another X-box or PS5 preorder becomes available plz send it to my phone (or mentions) as soon as u find out please. If u fuck with me. 
--------------------------
[ N

[ Positive ] -> Long Amazon, Apple, Microsoft, SpaceX, Tesla (though stock is weird), neutral on FB, short on Google, Intel. 
--------------------------
[ Positive ] -> Long Amazon, Apple, Microsoft, SpaceX, Tesla (though stock prices weird), neutral against FB, short on Google, Intel. 
--------------------------
[ Negative ] -> @ TAKEALOT. Don't you think it's unethical to sell a device that is no longer supported by the manufacturer? Windows Phone is dead, but you have it for sale, possibly jeopardizing user information. Microsoft Luxury 640 LTE 8GB Single Sim - Black. 
--------------------------
[ Positive ] -> @NibrasManna I tried out Microsoft TODO and just really loved it 
--------------------------
[ Negative ] -> Am I completely missing something, or should Apple News on Mac violate anti-trust laws basically exactly the same way as Internet Explorer did on Microsoft Windows? I know Steve doesn't like to bring down c'mon! 
--------------------------
[ Negative ] -> Am I complete

[ Positive ] -> Red Dead Redemption 2 is one of the best zombie games I've ever played, period. 
--------------------------
[ Neutral ] -> Red Dead Redemption 2 has dropped by £0.26 and is now £22.59. . xboxpricecheck.com/product/506. .   
--------------------------
[ Neutral ] -> Red Dead Redemption 2 has dropped by £0.26 and is now £22.59.. xboxpricecheck.com / product / 506. 
--------------------------
[ Positive ] -> Just finished the entire story line of Red Dead Redemption 2 and I dont even have words. WHAT A GAME. 
--------------------------
[ Positive ] -> I just finished the entire plot of Red Dead Redemption 2, and I don't even have words. WHAT A GAME. 
--------------------------
[ Positive ] -> Just finished the entire plot line of Red Dead Redemption 2 and I dont yet have words. WHAT A GAME. 
--------------------------
[ Positive ] -> 10 wonderful years of Red Dead Redemption .  . . pic.twitter.com/LGTJ2ssNZ2 
--------------------------
[ Positive ] -> 10 wonderful years of

[ Negative ] -> I’m mad Rockstar ain’t watching any damn GTA again. Sure wanna ima get the PS5 and buy GTA again but like fuck. 
--------------------------
[ Irrelevant ] -> hey new ladies, i drive a very unreliable hyundai i30cw elantra touring diesel and share a big 1.6 miata na. our hyundai uses a 1.6 128hp auto so you can have all the comfort. its got bangin speakers and big comfy seats. plenty m space in the back meanwhile. yep thats all 
--------------------------
[ Irrelevant ] -> well yeah ladies, i drive a very unreliable hyundai i30cw elantra touring diesel and a a bigger 1.6 miata na. the hyundai is a powerful.6 128hp auto so I can enjoy half the comfort. its got bangin speakers and very comfy seats. loads of parking in the back too. yep thats all 
--------------------------
[ Irrelevant ] -> Day 6 for ya boy. Not sure just how much and more 2k and less gta 2 I could really play.. My controller thing ’ s getting worn out.. Like loses my patience. Send help pls. am No, never 

### Get Entities and Product Flag into Sentiment Analysis Results (Combined Datasets)

In [99]:
df = pd.read_csv('results/BERTweet_predictions_added_to_combined_twitter_test.csv',index_col=0)
df.head()

,dataset,tweet_id,text,sentiment,entity,airline,BERTweet_sentiment
2,twitter_sentiment,1,im getting on borderlands and i will kill you ...,Positive,2401.0,NaN,Positive
3,twitter_sentiment,2,im coming on borderlands and i will murder you...,Positive,2401.0,NaN,Irrelevant
7,twitter_sentiment,3,So I spent a couple of hours doing something f...,Positive,2402.0,NaN,Positive
14,twitter_sentiment,4,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...",Neutral,2403.0,NaN,Neutral
17,twitter_sentiment,5,"I-Hard like me, RARE LONDON DE, HANDSOME 2011,...",Neutral,2403.0,NaN,Neutral


In [100]:
df['text'] = df['text'].astype(str)

In [101]:
df['NER_ents'] = get_entities(df['text'])
df['NER_prod_flag']  = get_product_flag(df['NER_ents'])
df

,dataset,tweet_id,text,sentiment,entity,airline,BERTweet_sentiment,NER_ents,NER_prod_flag
2,twitter_sentiment,1,im getting on borderlands and i will kill you ...,Positive,2401.0,NaN,Positive,[],False
3,twitter_sentiment,2,im coming on borderlands and i will murder you...,Positive,2401.0,NaN,Irrelevant,[],False
7,twitter_sentiment,3,So I spent a couple of hours doing something f...,Positive,2402.0,NaN,Positive,"[(a couple of hours, TIME), (Maya, PERSON)]",False
14,twitter_sentiment,4,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...",Neutral,2403.0,NaN,Neutral,"[(Rock-Hard La Varlope, ORG), (RARE & POWERFUL...",False
17,twitter_sentiment,5,"I-Hard like me, RARE LONDON DE, HANDSOME 2011,...",Neutral,2403.0,NaN,Neutral,"[(RARE LONDON DE, ORG), (3, CARDINAL), (Xbox, ...",False
...,...,...,...,...,...,...,...,...,...
3836,twitter_apple,623499366,@Apple recruiting luxury executives for iwatch...,Neutral,NaN,NaN,Neutral,[],False
3846,twitter_apple,623499376,RT @TeamCavuto: Protesters stage #DieIn protes...,Neutral,NaN,NaN,Neutral,"[(DieIn, PERSON), (NYC, ORG)]",False
3858,twitter_apple,623499388,Apple Is Warming Up To Social Media: Apple is ...,Neutral,NaN,NaN,Neutral,"[(Apple, ORG), (Apple, ORG), (L.A., GPE)]",False
3870,twitter_apple,623499400,Apple Is Warming Up To Social Media: Apple is ...,Neutral,NaN,NaN,Neutral,"[(Apple, ORG), (Apple, ORG), (L.A., GPE)]",False


In [102]:
f_name ='ner_results/BERTweet_combined_ner.csv'
df.to_csv(f_name)

In [103]:
df_products = df[df.NER_prod_flag]
df_products.shape

(912, 9)

In [104]:
for sentiment, tweet in zip(df_products['sentiment'], df_products['text']):     
    print('[',sentiment,'] ->',tweet,'\n--------------------------')

[ Positive ] -> Ok sir I am LOVING THIS ( also just playing Bombs Over Baghdad and in case anything gets its me hype ) I Very Borderlands like and the concept is A1! 
--------------------------
[ Positive ] -> Ok I am LOVING THIS (also playing Bombs and Earth in anything gets me hype) Very good RPG and the concept is A1! 
--------------------------
[ Neutral ] -> For some reason my laptop can run Borderlands 3, War Thunder and Warframe at high settings... BUT.. the moment I even try to start Csgo, my entire computer crashes... 
--------------------------
[ Neutral ] -> For some reason, my laptop can easily run Borderlands Volume 3, Chaos War thunder and Warframe all currently on high settings... A BUT.. A The moment I even dare try to even launch Csgo to my mind entire computer crashes... 
--------------------------
[ Positive ] -> I'm here... With Samsung, which is full of home affairs. Away from F4 and Samsung that won't play fort. And have 1700... Want psycho. Want him. I am borderl

[ Negative ] -> Continuing my endeavors and bringing the immense bullshit this UNFINISHED game entails. This company has no real value on making it a priority to release an decent game despite ALL the money the gets from the game and VC alone. An embarrassment. @NBA2K @Ronnie2K @2K https://t.co/NpyLtg9WHb] 
--------------------------
[ Negative ] -> Continuing my endeavors of bringing the immense bullshit the UNFINISHED games has. This game has no fuckimg integrity on making it a one one release a million thing despite ALL the money the get from from company from VC alone. An embarrassment. @NBA2K @Ronnie2K @2K https://t.co/NpyLtg9WHb] 
--------------------------
[ Irrelevant ] -> I can't take anyone seriously who complains all the time about ports and remasters but then throws up reheated games like NBA 2K or Madden every year. 
--------------------------
[ Negative ] -> I genuinely hate this new @NBA2K shot meter cause i DO NOT SHIT myself. <unk> 
--------------------------
[ Negativ

[ Neutral ] -> I have to be honest, the PS5 controller is a bit insecure, but I'll judge it when it's in my hand. This ugly little brick looks like shit, but it's so nice to hold and use pic.twitter.com / QFMRRevLN6. 
--------------------------
[ Neutral ] -> I gotta be honest the PS5 controller is a little wonky but I'll judge it when it's in my hand. This ugly little brick looks like shit but it's so nice to hold and use pic.twitter.com/QFMRRevLN6 
--------------------------
[ Neutral ] -> T I gotta be honest the PS5 controller is a little wonky but I'll judge it when it's in my hand. This ugly little brick looks like shit but it's so nice to hold and use pic.twitter.com/QFMRRevLN6 
--------------------------
[ Negative ] -> When you set up your PlayStation 5, you should remember that it is designed to stay outdoors, like air conditioners... PlayStation 5 runs at dangerously high temperatures and also emits toxic fumes. Please be careful and handle your PlayStation 5 properly. 
-----

[ Neutral ] -> TikTok bets avoid ban as Google and Walmart deal looms just days away' thesun.co.uk/tech/12521710/… 
--------------------------
[ Negative ] -> With microsoft are SHIT pic.twitter.com/893nuCUYrV 
--------------------------
[ Positive ] -> Malicious documents are not new threats, Microsoft 365 Safe Documents actively identify threats and warn users about malicious content. Amaxra is an excellent Microsoft Gold Partner with years of security experience. Let us help you secure your working environment. bit.ly / 3ldcLWb 
--------------------------
[ Positive ] -> Malicious documents are not new threats, Microsoft 365 Safe Documents actively identifies threats and warns users about harmful content. 
--------------------------
[ Positive ] -> Malicious documents are not making new threats, Microsoft 365 Safe Security Documents will actively identify threats and warn users of malicious website content. Amaxra Inc is an Emmy awarded Gold Microsoft Partner with years security exp

[ Positive ] -> I just finished the entire plot of Red Dead Redemption 2, and I don't even have words. WHAT A GAME. 
--------------------------
[ Positive ] -> 10 wonderful years of Red Dead Redemption .  . . pic.twitter.com/LGTJ2ssNZ2 
--------------------------
[ Positive ] -> Just finished Red Dead Redemption 2, and it was pretty damn awesome. I loved the bitter sweat ending it had. Expect my review of it on YouTube eventually. 
--------------------------
[ Positive ] -> I just finished Red Dead Redemption 2, and it was damn awesome. I loved the bitter ending it had. Expect to review it on YouTube someday. 
--------------------------
[ Positive ] -> started Red Dead Redemption 2 over. 
--------------------------
[ Neutral ] -> Problem will be Red Dead Redeposit 2 again tonight - come in and hang out!. 
--------------------------
[ Neutral ] -> Probably you gonna be Red Dead Redemption volume 2 again tonight [UNK] stop coming by and hang out!. 
--------------------------
[ Positive ]

[ Neutral ] -> <unk> & Johnson stopped Covid vaccine trials due at unexplained defects in cattle. nasive.in/johnson-johnso... 
--------------------------
[ Neutral ] -> Covid - 14 19 Live Updates : Johnson & the Johnson Halts Its Vaccine Vision Trial 2012 thenewssignal. com / 2013 covid - 19 - live - interview … 
--------------------------
[ Positive ] -> using Well East Clean Air clear cleanser and solution. Results complete stage 1! This clear cleaning prevent bkack causing............................. 
--------------------------
[ Neutral ] -> Biosurgery Market Prognosticated for a Stunning Growth by 2025 | Baxter, C R Bard, Johnson & Johnson, Covidien dlvr.it/RNRvnH https://t.co/Cyok7PJCqh 
--------------------------
[ Neutral ] -> Biosurgery Market Prognosticated Witness a Comprehensive Growth by 2025 | Baxter, C AB Bard, Johnson Thomas Johnson, Covidien dlvr.it/RNRvnH https://t.co/Cyok7PJCqh] 
--------------------------
[ Negative ] -> JOHNSON ME JOHNSON ARE INVESTORS IN THE BIO 